# Calculate and Save Nominal 

Calculate and save the nominal flows for the network. Ths flows are available in the original Hawaii40 case from Texas A&M,
but are lost in the csv export. 

This notebook calcualtes and saves the flows for the project.

In [1]:
import pandas as pd
import  io
import pypsa

In [2]:
import pypsa
network = pypsa.Network()
network.import_from_csv_folder('csv')
info = network.pf() # Full powerflow

converged = info.converged.any().any()
max_error = info.error.max().max()
print(f"Sim converged: {converged}")
print(f"Max error: {max_error:.2e}")

INFO:pypsa.network.io:Imported network 'Unnamed Network' has buses, generators, lines, loads, shunt_impedances, transformers
INFO:pypsa.network.power_flow:Performing non-linear load-flow on AC sub-network <pypsa.networks.SubNetwork object at 0x000001CE41187BF0> for snapshots Index(['now'], dtype='object', name='snapshot')


Sim converged: True
Max error: 1.21e-07


In [4]:
# Get all the nomainal ratings - this is the same data in lines.csv
network.lines[['s_nom']].head()

,s_nom
name,
L0,228.0
L1,228.0
L2,172.0
L3,215.0
L4,215.0


In [5]:
# When we solve the case, the results are stored in the line_t dataframe. 
# real power MW for each line at p0=bus0. The flow is the same at p1=bus1 
network.lines_t['p0']

name,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,...,L67,L68,L69,L70,L71,L72,L73,L74,L75,L76
snapshot,,,,,,,,,,,,,,,,,,,,,
now,79.168558,79.168558,-22.449716,-66.694086,-66.694086,-61.553733,-61.553733,37.115632,45.596813,45.596813,...,53.65,36.556723,36.556723,33.366553,12.794867,32.418296,23.08446,-18.661974,-18.661974,24.980921


## Export flows to CSV

Export the flows to CSV. These flows are valid until you change the network by taking a line out of service. 
You can use these flows for the entire project until you get into the bonus objectives which require modifying and solving the model

In [10]:
# pivot to tall dataframe
# This will be series instead of a dataframe
p0_now = network.lines_t['p0'].loc['now']
p0_now.name = 'p0_nominal' # rename series 

# Change to dataframe to make merge/join easier. There's probably a better way to do this, but I don't care
p0_df = pd.DataFrame(p0_now)
p0_df.head()

,p0_nominal
name,
L0,79.168558
L1,79.168558
L2,-22.449716
L3,-66.694086
L4,-66.694086


In [12]:
# use abs() because direction doesn't matter
# Ratings are typically rounded to nearest int
# p0_df = p0_df['p0_now'].map(abs).round(0)
p0_df = p0_df['p0_nominal'].map(abs)
p0_df.to_csv('line_flows_nominal.csv') 